# Reservoir monitoring time series application using the OPERA DSWx product
---

**This notebook serves gives example applications of reservoir monitoring over Lake Mead, NV from 2013-2023.**

In [ ]:
# notebook dependencies
import os
import math
import xarray as xr
import rasterio as rio
import rioxarray as rioxr
import hvplot.xarray
import geoviews as gv
import pyproj
from pyproj import Proj
import numpy as np
import pandas as pd
import geopandas as gpd
import holoviews as hv
import panel.widgets as pnw
import datetime
from datetime import datetime

from bokeh.models import FixedTicker
hv.extension('bokeh')

import warnings
warnings.filterwarnings('ignore')

os.environ['AWS_NO_SIGN_REQUEST'] = 'YES'

---
## **Data Information Input**

In the code cell below, the user should specify the:
* Dates of interest <br>
* Data directory<br>
* Band of interest<br>
* Path to shapefile to create mask<br><br>

**<font color='red'>Note: The cell below is the only code in the notebook that should be modified. </font>**

In [ ]:
# Access all provisional products from S3 bucket
data_dir = sorted(open("aux_files/T11SQA_manifest.txt","r").read().split("\n"))
for i,path in enumerate(data_dir):
   if path != '': # discard empty entries
       if path[91] == 'L': # Landsat 8 filenames
           data_dir[i] = path+path[32:101]+'_'
       if path[91] == 'S': #Sentinel-2 filenames
           data_dir[i] = path+path[32:102]+'_'
# Landsat filenames are 1 character shorter than Sentinel-2 filenames

# Extract date information from filename
dates = [path[57:65] for path in data_dir]

# Change this to the desired band for visualization
band = 'B01_WTR'

# Path to shapefile used to create mask of pixels close to Lake Mead
shapepath = 'aux_files/bufferlakebnds/bufferlakebnds.shp'

***

### **<font color='red'> -- Do not modify any of the code below -- </font>**

In [ ]:
# Functions

def file_dataframe(data_dir:str,datelist:str):
    '''
    Returns dataframe with data_dir to DSWx layers for a given list of dates.
        Parameters:
                data_dir (str): Working directory for each date
                datelist (str): List of dates
        Returns:
                dswx_file_df (Pandas DataFrame): List of data_dir to layer for each date
    '''
    fileDF = []
    for i,date in enumerate(dates):
        paths = f"{data_dir[i]}%s.tif"
        B01_WTR = paths%'B01_WTR'
        B02_BWTR = paths%'B02_BWTR'
        B03_CONF = paths%'B03_CONF'
        B04_DIAG = paths%'B04_DIAG.tif'
        B05_WTR_1 = paths%'B05_WTR-1.tif'
        B06_WTR_2 = paths%'B06_WTR-2.tif'
        B07_LAND = paths%'B07_LAND.tif'
        B08_SHAD = paths%'*B08_SHAD.tif'
        B09_CLOUD = paths%'*B09_CLOUD.tif'
        B10_DEM = paths%'DEM.tif'
        fileDF.append([date,B01_WTR,B02_BWTR,B03_CONF,B04_DIAG,B05_WTR_1,B06_WTR_2,B07_LAND,B08_SHAD,B09_CLOUD,B10_DEM])
    fileDF = pd.DataFrame(fileDF,columns = ['Date', 'B01_WTR', 'B02_BWTR', 'B03_CONF', 'B04_DIAG', 'B05_WTR_1', 'B06_WTR_2', \
        'B07_LAND', 'B08_SHAD', 'B09_CLOUD', 'B10_DEM']).astype('string')
    return fileDF

def stack_layers(files:str,datelist:str):
    '''
    Returns geocube with one band over time stacked into one multi-dimensional array.
            Parameters:
                    files (str): Paths to band for each date
                    datelist (list): List of dates
            Returns:
                    layerStack (xarray Dataset): Geocube with band stacked in time
                    crs (int): Coordinate Reference System corresponding to band
    '''
    layerStack = []; layerS = []; layerStack_ = [];
    for i,d in enumerate(datelist):
        time = datetime.strptime(d,'%Y%m%d')
        if i == 0:
            layerStack_ = xr.open_rasterio(files[i])
            crs = pyproj.CRS.to_epsg(pyproj.CRS.from_proj4(layerStack_.crs))
            layerStack = layerStack_.squeeze(drop=True)
            layerStack = layerStack.to_dataset(name='z')
            layerStack.coords['time'] = np.array(time)
            layerStack = layerStack.rename({'x':'longitude', 'y':'latitude'})
            layerStack = layerStack.expand_dims(dim='time')
        else:
            layerS = xr.open_rasterio(files[i])
            layerS = layerS.squeeze(drop=True)
            layerS = layerS.to_dataset(name='z')
            layerS.coords['time'] = np.array(time)
            layerS = layerS.rename({'x':'longitude', 'y':'latitude'})
            layerS = layerS.expand_dims(dim='time')
            layerStack = xr.concat([layerStack,layerS], dim='time')
    return layerStack, crs

def buffer_mask(shapefile:str):
    '''
    Returns masked data based on buffered shapefile.
            Parameters:
                    shapefile (str): Path to buffered shapefile
            Returns:
                    masked (xarray Dataset): Geocube of masked data
    '''
    shp = gpd.read_file(shapefile)
    maskedData = data.rio.clip(shp.geometry)
    maskedData = maskedData.where(maskedData['z'] != 255.)
    return maskedData

def compute_area(data,dates):
    '''
    Returns area for each layer value for each date.
            Parameters:
                    data (xarray Dataset): Band of masked geocube
                    dates (xarray Dataset): Dates in datetime format
            Returns:
                    pixelArea (xarray Dataset): Dataset of area of each layer value
    '''
    nd = len(data)
    water = np.empty(nd)
    partial_water = np.empty(nd)
    clouds = np.empty(nd)
    
    for i in range(nd):
        df = data[i]
        water[i] = np.count_nonzero(df==1)*900
        partial_water[i] = np.count_nonzero(df==2)*900
        clouds[i] = (np.count_nonzero(df==9)/np.count_nonzero(df))*100

    pixelArea = xr.Dataset(data_vars={'Water': ('time',water), 'Partial Water': ('time',partial_water), \
        'Clouds': ('time',clouds)}, coords={'time': dates})
    return pixelArea

def compute_occurrence(datelist:str,data):
    '''
    Caluclates surface water occurrence percent by dividing sum of 
    water detection by sum of valid occurrences per month
            Parameters:
                    data (xarray Dataset): Masked geocube
            Returns:
                    percent (xarray Dataset): Geocube of water percent 
    '''
    years = []
    for date in datelist:
        year_ = date[0:4]
        years.append(year_)
    
    years = sorted(list(set(years)))

    percentCube = []; percentCube_ = []; percent = [];

    data = data.where(data!=9) # mask out cloud pixels

    for year in years:
        dy = data.isel(time=data.time.dt.year.isin([int(year)]))
        for i in range(1,12):
            dm = dy.isel(time=dy.time.dt.month.isin([i]))
            count = np.sum(dm.z==1,axis=0)+np.sum(dm.z==2,axis=0)
            total = np.count_nonzero(~np.isnan(dm.z),axis=0)
            if (year == years[0] and i==1):
                percentCube = (count/total)*100
                percentCube = percentCube.to_dataset(name='z')
                percentCube.coords['month'] = year+'-'+str(i)
                percentCube = percentCube.expand_dims(dim='month')
            else:
                percent = (count/total)*100
                percent = percent.to_dataset(name='z')
                percent.coords['month'] = year+'-'+str(i)
                percent = percent.expand_dims(dim='month')
                percentCube = xr.concat([percentCube,percent], dim='month')
                
    percentCube = np.mean(percentCube.z,axis=0)
    return percentCube
    

### **<font color='red'> -- Do not modify any of the code above -- </font>**

---
## **1. Prepare the Geocube: Create the file dataframe, multidimensional dataset, and mask**

In [ ]:
# Create dataframe of paths to bands for each date
dswx_file_df = file_dataframe(data_dir,dates)
# Inspect the dataframe
dswx_file_df

In [ ]:
# Stack dates to create a multidimensional "geocube" for chosen band
data, crs = stack_layers(dswx_file_df[band], dates)
# Inspect the geocube dataset created
data

Only include pixels near Lake Meak by creating mask from buffered shapefile based on a [2003 USGS shapefile](https://pubs.usgs.gov/of/2009/1150/gis/basemap/lakebndsmeta.htm). This shapefile is created using the optional [prep_shapefile](link) notebook provided in this repository.

In [ ]:
# Create a masked geocube
masked_data = buffer_mask(shapepath)
# Inspect the masked dataset
masked_data

---
## **2. Visualize surface water extent of Lake Mead with all available dates**

In [ ]:
# Create a basemap
base = gv.tile_sources.EsriImagery.opts(width=1000, height=1000, padding=0.1)

dswxPalette = ["#2000ff","#04fc04","#ffffff","#ffffff","#ffffff","#ffffff","#ffffff","#ffffff","#7f7f7f"]

masked_edit = masked_data.where(masked_data['z'] != 4.) # Flag 4 has a lot of false positives

masked_data_z = masked_edit.z.where(masked_edit.z>0)

masked_data_slider = masked_data_z.interactive.sel(time=pnw.DiscreteSlider).hvplot(x='longitude', 
                                                                                       y='latitude', 
                                                                                       crs=crs, 
                                                                                       kind='image', 
                                                                                       rasterize=True, 
                                                                                       dynamic=True,
                                                                                       cmap=dswxPalette, 
                                                                                       aspect='equal', 
                                                                                       frame_width=600, 
                                                                                       frame_height=600).opts(active_tools=['wheel_zoom'],
                                                                                                              xlabel='Longitude', 
                                                                                                              ylabel='Latitude',  
                                                                                                              clim=(1,9), 
                                                                                                              colorbar_opts={'ticker': FixedTicker(ticks=[0, 1, 2, 9])}, 
                                                                                                              alpha=0.9)
masked_data_slider * base

**Layer Values:**<br> 
* **0:** Not Water – an area with valid reflectance data that is not open water (class 1), partial surface water (class 2), or
cloud/cloud shadow (class 9). Masking can result in “not water” (class 0) where land cover masking is applied<br>
* **1:** Open Water – an area that is entirely water and unobstructed to the sensor, including obstructions by vegetation, terrain,
and buildings <br>
* **2:** Partial Surface Water – an area that is at least 50% and less than 100% open water. This may be referred to as “subpixel
inundation” when referring to a pixel’s area. Examples include inundated sinkholes, floating vegetation, and pixels bisected by
coastlines <br> 
* **9:** Cloud/Cloud Shadow – an area identified as cloud, cloud shadow, or snow/ice according to input quality assurance (QA)
data <br>
* **255:** Fill value (no data)  <br> 

---
## **3. Calculate water area over time**

In [ ]:
# Calculate area of each layer value
area = compute_area(masked_data.z,masked_data.time)
# Inspect the new dataset
area

We can plot water area vs. time in two ways. Below we utilize `matplotlib` subplots.

In [ ]:
n = 25

fig, (ax1,ax2,ax3) = plt.subplots(3,sharex=True)
fig.set_size_inches(30,15)
fig.tight_layout()

p1,=ax1.plot(area['time'],area['Water'],color='black',linewidth=2.0)
p1,=ax1.plot(area['time'],area['Water'],color="#2000ff")
p2,=ax2.plot(area['time'],area['Partial Water'],color='black',linewidth=2.0)
p2,=ax2.plot(area['time'],area['Partial Water'],color="#04fc04")
p3,=ax3.plot(area['time'],area['Clouds'],color='black',linewidth=2.0)
p3,=ax3.plot(area['time'],area['Clouds'],color="#7f7f7f")

ax1.tick_params(axis='both', labelsize=n, direction='out', length=6, width=3)
ax2.tick_params(axis='both', labelsize=n,direction='out', length=6, width=3)
ax3.tick_params(axis='both', labelsize=n, direction='out', length=6, width=3)

We can also create a more dynamic plot using `hvplot`.

In [ ]:
area.hvplot.line(x='time', y=['Water','Partial Water','Clouds'], value_label = 'Area (m^2)',
                width=1000, height=300, subplots=True, shared_axes=False).cols(1).opts(title=f"Area of Masked Pixels",fontsize={'title': 16, 'labels': 14, 'xticks': 6, 'yticks': 12})

---
## **4. Calculate and visualize water occurrence**

In [ ]:
# Calculate area of each layer value
occurrence = compute_occurrence(dates,masked_data)
# Inspect the new dataset
occurrence

In [ ]:
# Create a basemap
base = gv.tile_sources.EsriImagery.opts(width=1000, height=1000, padding=0.1)

occurrence.where(occurrence>0).hvplot.image(x='longitude', 
                          y='latitude', 
                          crs=crs, 
                          rasterize=True, 
                          dynamic=True, 
                          aspect='equal', 
                          frame_width=500, 
                          frame_height=500, 
                          cmap='plasma', 
                          clim=(0,100), alpha=0.8).opts(title=f"Water Occurrence", xlabel='Longitude', ylabel='Latitude') * base